In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import json
from json import JSONEncoder

In [2]:
baseUrl = "https://www.vlr.gg"

class HeadToHeadData():
    def __init__(self):
        self.Date = ""
        self.Team1Score = 0
        self.Team2Score = 0

class PastMatchesData():
    def __init__(self):
        self.Team2 = ""
        self.OurTeamScore = 0
        self.Team2Score = 0
        self.Date = ""
        
class PlayerData():
    def __init__(self):
        self.Name = ""
        self.Team = ""
        self.ACS = 0
        self.Kills = 0
        self.Deaths = 0
        self.Assists = 0
        self.KAST = 0
        self.ADR = 0
        self.HeadshotPercent = 0
        self.FirstKills = 0
        self.FirstDeaths = 0

class MapData():
    def __init__(self):
        self.ListOfPlayerDataTeam1 = []
        self.ListOfPlayerDataTeam2 = []
        self.ListOfRounds = []
        self.MapName = "" #All if it is overall stats
        self.MapNumber = 0 #0 if overall stats
        self.Team1Score = 0
        self.Team1Attack = 0
        self.Team2Score = 0
        self.Team2Attack = 0
        self.Pick = ""
        
class MatchData():
    def __init__(self):
        self.Date = ""
        self.Team1 = ""
        self.Team1Score = 0
        self.Team2 = ""
        self.Team2Score = 0
        
        self.ListOfMapData = []
        
        self.ListOfHeadToHead = []
        self.Team1PastMatches = []
        self.Team2PastMatches = []
        
class EmployeeEncoder(JSONEncoder):
        def default(self, o):
            return o.__dict__

In [3]:
response = requests.get("https://www.vlr.gg/event/matches/1015/valorant-champions-2022/?series_id=all&group=completed")

In [4]:
response.status_code

200

In [5]:
eventSoup = BeautifulSoup(response.text)

In [6]:
eventSoup.title.text

'\n\t\t\tValorant Champions 2022: Schedule and Results | Valorant Event | VLR.gg\t\t'

In [7]:
def MatchExactClassName(tag):
    return tag.has_attr('class') and tag['class'] == ['wf-card']

In [8]:
daysMatches = eventSoup.body.find_all(MatchExactClassName)

In [9]:
matchId = 0

for dayMatches in daysMatches:
    matchesOfTheDay = dayMatches.find_all('a')
    for match in matchesOfTheDay:
        matchId = matchId + 1
        print("Match: " + str(matchId))
        matchSoup = BeautifulSoup(requests.get(baseUrl + match['href']).text)
        
        fileName = str(matchId) + " " + match['href'].split("/")[-1] + ' '.join(matchSoup.find_all("div", {"class": "match-header-date"})[0].text.split())
        
        matchData = MatchData()
        matchData.Date = ' '.join(matchSoup.find_all("div", {"class": "match-header-date"})[0].text.split())
        matchData.Team1 = ' '.join(matchSoup.find_all("div", {"class": "wf-title-med"})[0].text.split())
        matchData.Team2 = ' '.join(matchSoup.find_all("div", {"class": "wf-title-med"})[1].text.split())
        matchData.Team1Score = int(' '.join(matchSoup.find_all("div", {"class": "js-spoiler"})[0].find_all("span")[0].text.split()))
        matchData.Team2Score = int(' '.join(matchSoup.find_all("div", {"class": "js-spoiler"})[0].find_all("span")[-1].text.split()))
        
        ############# Head to Head Matches section
        h2hSection = matchSoup.find_all("a", {"class": "mod-h2h"})
        
        for h2h in h2hSection:
            h2hData = HeadToHeadData()
            
            h2hData.Team1Score = int(h2h.find_all("span", {"class": "rf"})[0].text)
            h2hData.Team2Score = int(h2h.find_all("span", {"class": "ra"})[0].text)
            h2hData.Date = ' '.join(h2h.find_all("div", {"class": "match-h2h-matches-date"})[0].text.split())
            
            matchData.ListOfHeadToHead.append(h2hData)
        
        ############# Match History Section
        matchHistorySections = matchSoup.find_all("div", {"class": "match-histories"})
        
        #Team1     matchHistorySections[0]
        for matchHistory in matchHistorySections[0].find_all("a", {"class": "match-histories-item"}):
            pastMatchesData1 = PastMatchesData()
            
            pastMatchesData1.OurTeamScore = int(matchHistory.find_all("span", {"class": "rf"})[0].text)
            pastMatchesData1.Team2Score = int(matchHistory.find_all("span", {"class": "ra"})[0].text)
            pastMatchesData1.Team2 = matchHistory.find_all("span", {"class": "match-histories-item-opponent-name"})[0].text
            pastMatchesData1.Date = ' '.join(matchHistory.find_all("div", {"class": "match-histories-item-date"})[0].text.split())
            
            matchData.Team1PastMatches.append(pastMatchesData1)
            
        #Team2     matchHistorySections[1]
        for matchHistory in matchHistorySections[1].find_all("a", {"class": "match-histories-item"}):
            pastMatchesData2 = PastMatchesData()
            
            pastMatchesData2.OurTeamScore = int(matchHistory.find_all("span", {"class": "rf"})[0].text)
            pastMatchesData2.Team2Score = int(matchHistory.find_all("span", {"class": "ra"})[0].text)
            pastMatchesData2.Team2 = matchHistory.find_all("span", {"class": "match-histories-item-opponent-name"})[0].text
            pastMatchesData2.Date = ' '.join(matchHistory.find_all("div", {"class": "match-histories-item-date"})[0].text.split())
            
            matchData.Team2PastMatches.append(pastMatchesData2)
            
        
        ############# Map Data
        mapsSoup = matchSoup.find_all("div", {"class": "vm-stats-game"})
        
        mapNumber = 1
        for mapSoup in mapsSoup:
            mapData = MapData()
            
            ## Map specific details
            if(mapSoup['data-game-id'] != 'all'):
                mapData.MapNumber = mapNumber
                mapData.MapName = mapSoup.find_all("div", {"class": "map"})[0].span.text.split()[0]
                mapData.Team1Score = int(mapSoup.find_all("div", {"class": "score"})[0].text)
                mapData.Team2Score = int(mapSoup.find_all("div", {"class": "score"})[1].text)
                mapData.Team1Attack = int(mapSoup.find_all("span", {"class": "mod-t"})[0].text)
                mapData.Team2Attack = int(mapSoup.find_all("span", {"class": "mod-t"})[1].text)
                
                if(len(mapSoup.find_all("div", {"class": "map"})[0].find_all("span")) == 1):
                    mapData.Pick = "Decider"
                else:
                    mapData.Pick = "Team " + mapSoup.find_all("div", {"class": "map"})[0].span()[0]['class'][1][-1]
                
                scoreSoupList = mapSoup.find_all("div", {"class": "rnd-currscore"})
                for scoreSoup in scoreSoupList:
                    if(len(scoreSoup.text.split()) > 0):
                        mapData.ListOfRounds.append(scoreSoup.text.split()[0].split("-"))

                mapNumber = mapNumber + 1
            else:
                mapData.MapName = 'all'
                mapData.MapNumber = 0
            
            ## Player stats
            
            playerDataSoup = mapSoup.find_all("table", {"class": "wf-table-inset mod-overview"})
                    
            # Team1    playerDataSoup[0]
            team1PlayersSoup = playerDataSoup[0].tbody.find_all("tr")
            for playerSoup in team1PlayersSoup:
                playerData = PlayerData()
                
                playerData.Name = playerSoup.find_all("div", {"class": "text-of"})[0].text.split()[0]
                playerData.Team = playerSoup.find_all("div", {"class": "ge-text-light"})[0].text.split()[0]
                
                statsList = playerSoup.find_all("td", {"class": "mod-stat"})
                
                playerData.ACS = [int(statsList[0].find_all("span", {"class": "mod-both"})[0].text), int(statsList[0].find_all("span", {"class": "mod-t"})[0].text), int(statsList[0].find_all("span", {"class": "mod-ct"})[0].text)]
                playerData.Kills = [int(statsList[1].find_all("span", {"class": "mod-both"})[0].text), int(statsList[1].find_all("span", {"class": "mod-t"})[0].text), int(statsList[1].find_all("span", {"class": "mod-ct"})[0].text)]
                playerData.Deaths = [int(statsList[2].find_all("span", {"class": "mod-both"})[0].text), int(statsList[2].find_all("span", {"class": "mod-t"})[0].text), int(statsList[2].find_all("span", {"class": "mod-ct"})[0].text)]
                playerData.Assists = [int(statsList[3].find_all("span", {"class": "mod-both"})[0].text), int(statsList[3].find_all("span", {"class": "mod-t"})[0].text), int(statsList[3].find_all("span", {"class": "mod-ct"})[0].text)]
                playerData.KAST = [statsList[5].find_all("span", {"class": "mod-both"})[0].text, statsList[5].find_all("span", {"class": "mod-t"})[0].text, statsList[5].find_all("span", {"class": "mod-ct"})[0].text]
                playerData.ADR = [int(statsList[6].find_all("span", {"class": "mod-both"})[0].text), int(statsList[6].find_all("span", {"class": "mod-t"})[0].text), int(statsList[6].find_all("span", {"class": "mod-ct"})[0].text)]
                playerData.HeadshotPercent = [statsList[7].find_all("span", {"class": "mod-both"})[0].text, statsList[7].find_all("span", {"class": "mod-t"})[0].text, statsList[7].find_all("span", {"class": "mod-ct"})[0].text]
                playerData.FirstKills = [int(statsList[8].find_all("span", {"class": "mod-both"})[0].text), int(statsList[8].find_all("span", {"class": "mod-t"})[0].text), int(statsList[8].find_all("span", {"class": "mod-ct"})[0].text)]
                playerData.FirstDeaths = [int(statsList[9].find_all("span", {"class": "mod-both"})[0].text), int(statsList[9].find_all("span", {"class": "mod-t"})[0].text), int(statsList[9].find_all("span", {"class": "mod-ct"})[0].text)]
                
                mapData.ListOfPlayerDataTeam1.append(playerData)
            
            # Team2    playerDataSoup[1]
            team2PlayersSoup = playerDataSoup[1].tbody.find_all("tr")
            for playerSoup in team2PlayersSoup:
                playerData = PlayerData()
                
                playerData.Name = playerSoup.find_all("div", {"class": "text-of"})[0].text.split()[0]
                playerData.Team = playerSoup.find_all("div", {"class": "ge-text-light"})[0].text.split()[0]
                
                statsList = playerSoup.find_all("td", {"class": "mod-stat"})
                
                playerData.ACS = [int(statsList[0].find_all("span", {"class": "mod-both"})[0].text), int(statsList[0].find_all("span", {"class": "mod-t"})[0].text), int(statsList[0].find_all("span", {"class": "mod-ct"})[0].text)]
                playerData.Kills = [int(statsList[1].find_all("span", {"class": "mod-both"})[0].text), int(statsList[1].find_all("span", {"class": "mod-t"})[0].text), int(statsList[1].find_all("span", {"class": "mod-ct"})[0].text)]
                playerData.Deaths = [int(statsList[2].find_all("span", {"class": "mod-both"})[0].text), int(statsList[2].find_all("span", {"class": "mod-t"})[0].text), int(statsList[2].find_all("span", {"class": "mod-ct"})[0].text)]
                playerData.Assists = [int(statsList[3].find_all("span", {"class": "mod-both"})[0].text), int(statsList[3].find_all("span", {"class": "mod-t"})[0].text), int(statsList[3].find_all("span", {"class": "mod-ct"})[0].text)]
                playerData.KAST = [statsList[5].find_all("span", {"class": "mod-both"})[0].text, statsList[5].find_all("span", {"class": "mod-t"})[0].text, statsList[5].find_all("span", {"class": "mod-ct"})[0].text]
                playerData.ADR = [int(statsList[6].find_all("span", {"class": "mod-both"})[0].text), int(statsList[6].find_all("span", {"class": "mod-t"})[0].text), int(statsList[6].find_all("span", {"class": "mod-ct"})[0].text)]
                playerData.HeadshotPercent = [statsList[7].find_all("span", {"class": "mod-both"})[0].text, statsList[7].find_all("span", {"class": "mod-t"})[0].text, statsList[7].find_all("span", {"class": "mod-ct"})[0].text]
                playerData.FirstKills = [int(statsList[8].find_all("span", {"class": "mod-both"})[0].text), int(statsList[8].find_all("span", {"class": "mod-t"})[0].text), int(statsList[8].find_all("span", {"class": "mod-ct"})[0].text)]
                playerData.FirstDeaths = [int(statsList[9].find_all("span", {"class": "mod-both"})[0].text), int(statsList[9].find_all("span", {"class": "mod-t"})[0].text), int(statsList[9].find_all("span", {"class": "mod-ct"})[0].text)]
                
                mapData.ListOfPlayerDataTeam2.append(playerData)
            
            
            matchData.ListOfMapData.append(mapData)
            
            
        #json_object = json.dumps(EmployeeEncoder().encode(matchData))
        json_object = json.dumps(matchData, indent=4, cls=EmployeeEncoder)
        with open(fileName+".json", "w") as outfile:
            outfile.write(json_object)

Match: 1
Match: 2
Match: 3
Match: 4
Match: 5
Match: 6
Match: 7
Match: 8
Match: 9
Match: 10
Match: 11
Match: 12
Match: 13
Match: 14
Match: 15
Match: 16
Match: 17
Match: 18
Match: 19
Match: 20
Match: 21
Match: 22
Match: 23
Match: 24
Match: 25
Match: 26
Match: 27
Match: 28
Match: 29
Match: 30
Match: 31
Match: 32
Match: 33
Match: 34
